In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
import re
import json
# from sklearn import cross_validation as cv
from collections import Counter
# from tensorflow.python.ops import math_ops
import matplotlib.pyplot as plt
import time
import datetime

In [2]:
article_title = [ 'id', 'feedname', 'url', 'orig_html', 'title', 'bodytext', 'relevant', 'created', 'last_modified', 'news_sourse_id', 'author']
article = pd.read_csv('./newsarticles_article.csv', header=None, names=article_title, dtype={'orig_html': str, 'author': str})

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
article.head()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_sourse_id,author
0,122140,A,http://abclocal.go.com/wls/story?section=news/...,NaN,Ventra cards available for all users Monday | ...,"![][1]\n\nSeptember 8, 2013 (CHICAGO) (WLS) --...",f,2013-09-09 14:09:06.165191+00,2013-09-23 14:38:23.080001+00,98,NaN
1,122141,A,http://abclocal.go.com/wls/story?section=news/...,NaN,Chicago Bears win season opener over Cincinnat...,"![AP][1]\n\nSeptember 8, 2013 (CHICAGO) -- Fa...",f,2013-09-09 14:09:07.943208+00,2013-09-23 14:38:07.687984+00,98,NaN
2,122142,F,http://www.myfoxchicago.com/story/23377975/she...,NaN,"Sheriff: Inmate stabs sheriff's deputy, escape...",* ### Sheriff: Inmate stabs sheriff's deput...,f,2013-09-09 14:09:51.837171+00,2014-02-25 22:23:20.605934+00,109,NaN
3,122143,C,http://www.beachwoodreporter.com/tv/24_hours_w...,NaN,24 Hours With Al Jazeera America,Returning news from exile.\n\n**Midnight:** _R...,f,2013-09-09 14:11:53.733228+00,2013-09-23 14:38:23.088423+00,99,NaN
4,122144,C,http://www.beachwoodreporter.com/music/the_wee...,NaN,The Weekend In Chicago Rock,You shoulda been there.\n\n**1.** Chance The R...,f,2013-09-09 14:11:54.776239+00,2013-09-23 14:38:23.093802+00,99,NaN


In [4]:
article['relevant'] = article['relevant'] == 't'
article.rename(columns={'id': 'article_id'}, inplace=True)
article.head()

,article_id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_sourse_id,author
0,122140,A,http://abclocal.go.com/wls/story?section=news/...,NaN,Ventra cards available for all users Monday | ...,"![][1]\n\nSeptember 8, 2013 (CHICAGO) (WLS) --...",False,2013-09-09 14:09:06.165191+00,2013-09-23 14:38:23.080001+00,98,NaN
1,122141,A,http://abclocal.go.com/wls/story?section=news/...,NaN,Chicago Bears win season opener over Cincinnat...,"![AP][1]\n\nSeptember 8, 2013 (CHICAGO) -- Fa...",False,2013-09-09 14:09:07.943208+00,2013-09-23 14:38:07.687984+00,98,NaN
2,122142,F,http://www.myfoxchicago.com/story/23377975/she...,NaN,"Sheriff: Inmate stabs sheriff's deputy, escape...",* ### Sheriff: Inmate stabs sheriff's deput...,False,2013-09-09 14:09:51.837171+00,2014-02-25 22:23:20.605934+00,109,NaN
3,122143,C,http://www.beachwoodreporter.com/tv/24_hours_w...,NaN,24 Hours With Al Jazeera America,Returning news from exile.\n\n**Midnight:** _R...,False,2013-09-09 14:11:53.733228+00,2013-09-23 14:38:23.088423+00,99,NaN
4,122144,C,http://www.beachwoodreporter.com/music/the_wee...,NaN,The Weekend In Chicago Rock,You shoulda been there.\n\n**1.** Chance The R...,False,2013-09-09 14:11:54.776239+00,2013-09-23 14:38:23.093802+00,99,NaN


In [5]:
usercoding_title = ['id', 'date', 'relevant', 'article_id', 'user_id', 'locations']
usercoding = pd.read_csv('./newsarticles_usercoding.csv', header=None, names=usercoding_title)

In [6]:
usercoding.head()

,id,date,relevant,article_id,user_id,locations
0,1,2017-04-30 23:51:50.229747+00,f,1,NaN,[]
1,2,2017-04-30 23:51:50.238775+00,f,2,NaN,[]
2,3,2017-04-30 23:51:50.243216+00,f,3,NaN,[]
3,4,2017-04-30 23:51:50.247388+00,f,4,NaN,[]
4,5,2017-04-30 23:51:50.251488+00,f,5,NaN,[]


In [7]:
usercoding.locations.unique()

array(['[]',
       '[{"start":211,"end":227,"text":"3252 W. 63rd St."},{"start":674,"end":688,"text":"600 W. 37th St"}]',
       '[{"start":189,"end":221,"text":"500 block of North State Street,"},{"start":73,"end":88,"text":"Near North Side"}]',
       ...,
       '[{"start":269,"end":288,"text":"U.S. Cellular Field"},{"start":2233,"end":2247,"text":"Glendale, Ariz"}]',
       '[{"start":71,"end":96,"text":"South and Near West sides"},{"start":181,"end":210,"text":"300 block of East 47th Street"},{"start":218,"end":242,"text":"Bronzeville neighborhood"},{"start":498,"end":512,"text":"Near West Side"},{"start":589,"end":622,"text":"1800 block of West Maypole Avenue"}]',
       '[{"start":83,"end":93,"text":"Englew ood"},{"start":114,"end":124,"text":"South Side"},{"start":188,"end":222,"text":"5500 block of South Emerald Avenue"}]'], dtype=object)

In [8]:
# usercoding.locations = usercoding.locations.apply(lambda x: json.loads(x))

In [9]:
usercoding.locations


0                                                        []
1                                                        []
2                                                        []
3                                                        []
4                                                        []
5                                                        []
6                                                        []
7                                                        []
8                                                        []
9                                                        []
10                                                       []
11                                                       []
12                                                       []
13                                                       []
14                                                       []
15                                                       []
16                                      

In [16]:
def getAddressText(json_string):
    json_address = json.loads(json_string)
    for address in json_address:
        print(address['text'])

In [17]:
usercoding.locations.apply(getAddressText)

3252 W. 63rd St.
600 W. 37th St
500 block of North State Street,
Near North Side
3500 block of West Congress
East Garfield Park
800 block of North Springfield Avenue
West Humboldt Park 
3500 block of West Congress Parkway
Garfield Park
River North
South Shore
Fuller Park
West Humboldt Park
Near West Side.
Park Manor
600 block of East 71st Street
Back of the Yards
Garfield Park
700 block of North Lawndale Avenue
200 block of West 47th Stree
71st Street in Park Manor,
8200 block of South Emerald,
West Englewood
Auburn Gresham
3000 block of West Division
West Side Humboldt Park neighborhood
945 W. Belmont Ave
Pilsen
2200 block of West 18th Place 
1900 block of South Shelby Court
North Side
4301 W. Chicago Ave.
West Town
1300 W. Jackson
1010 S. Clinton 
1338 S. Clinton,
West Garfield Park
South Side
West Side
Lake View
North Side
945 W. Belmont Ave.
West Garfield Park
6200 block of South Indiana
300 block of East 57th Street
Washington Park
South Side
West Garfield Park
4301 West Chicago A

 1800 block of South Allport Street,
Dan Ryan Expressway in Chinatown. 
1800 block of North Keeler Avenue 
e near O'Hare
 200 block of North Sacramento Boulevard
 12800 block of South Lowe Avenue
100 block of North Harding Street 
 West Rogers Park neighborhoo
Northwest Side. 
2600 block of West Peterson
Northwest Side 
 2958 N. Kenneth Ave.
Roseland neighborhood on the Far South Side. 
11200 block of South Wentworth,
 South Side 
Wentworth Gardens near 37th Street and Princeton Avenue 
 West Side Austin neighborhood,
 corner of Chicago and Lockwood,
1000 block of West Wilson in Uptown,
 8000 block of South Coles, 
 West Englewood’s 5900 block of South Loomis 
4600 block of South Kedzie
Brighton Park on the Southwest Side.
 South Side Bronzeville neighborhood,
near routes 59 and 173 in Antioch
University Village neighborhood, 
1300 block of West Hastings
Grand Boulevard 
Bronzeville neighborhood on the South Side 
 400 block of East Oakwood Boulevard
 2800 block of West Balmoral Avenue

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
234489    None
234490    None
234491    None
234492    None
234493    None
234494    None
234495    None
234496    None
234497    None
234498    None
234499    None
234500    None
234501    None
234502    None
234503    None
234504    None
234505    None
234506    None
234507    None
234508    None
234509    None
234510    None
234511    None
234512    None
234513    None
234514    None
234515    None
234516    None
234517    None
234518    None
Name: locations, Length: 234519, dtype: object